In [21]:
#Se cargan las bibliotecas necesarias 4
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from skimage. feature import hog
import matplotlib.image as mpimg



In [18]:
import glob

pedestrian = glob.glob("data_svm/Pedestrian/*.jpg")
no_pedestrian= glob.glob("data_svm/nonPedestrian/*.jpg")

In [19]:
len(pedestrian)


0

In [20]:
len(no_pedestrian)

0

In [ ]:
#Se muestra imagen con un peaton
img_color= cv2.imread(pedestrian[10])
plt.imshow(img_color)

In [ ]:
img_gray = cv2.cvtColor(img_color,cv2.COLOR_BAYER_BG2GRAY)
plt.imshow(img_gray,cmap ="gray")

In [ ]:
features,hog_img = hog(img_gray,
                       orientations =11,
                       pixels_per_cell= (16,16),
                       cells_per_block=(2,2),
                       transform_sqrt= False,
                       visualize = True,
                       feature_vector= True)
                      

In [ ]:
features.shape

In [ ]:
hog_img.shape

In [ ]:
plt.imShow(hog_img,cmap = 'gray')

In [ ]:
pedestrian_hog_accum = []

for i  in pedestrian:
  img_color = mpimg.imread(i)
  img_gray = cv2.cvtColor(img_color,cv2.COLOR_BAYER_BG2GRAY)

  pedestrian_hog_feature,pedestrian_hog_img  = hog(img_gray,
                                              orientations =11,
                                              pixels_per_cell= (16,16),
                                              cells_per_block=(2,2),
                                              transform_sqrt= False,
                                              visualize = True,
                                              feature_vector= True)

  pedestrian_hog_accum. append (pedestrian_hog_feature)

In [ ]:
X_pedestrian = np.vstack(pedestrian_hog_accum).astype(np.float64)
Y_pedestrian = np.ones(len(X_pedestrian))

In [ ]:
X_pedestrian.shape

In [ ]:
Y_pedestrian.shape

In [ ]:
nopedestrian_hog_accum = []

for i  in no_pedestrian:
  img_color = mpimg.imread(i)
  img_gray = cv2.cvtColor(img_color,cv2.COLOR_BAYER_BG2GRAY)

  nopedestrian_hog_feature,nonpedestrian_hog_img  = hog(img_gray,
                                              orientations =11,
                                              pixels_per_cell= (16,16),
                                              cells_per_block=(2,2),
                                              transform_sqrt= False,
                                              visualize = True,
                                              feature_vector= True)

  nopedestrian_hog_accum. append (nopedestrian_hog_feature)

In [ ]:
X_nopedestrian = np.vstack(nopedestrian_hog_accum).astype(np.float64)
Y_nopedestrian = np.zeros(len(X_nopedestrian))

In [ ]:
X_nopedestrian.shape

In [ ]:
Y_nopedestrian.shape

In [ ]:
X = np.vstack((X_pedestrian,X_nopedestrian))
X.shape

In [ ]:
Y = np.vstack((Y_pedestrian,Y_nopedestrian))
Y.shape

In [ ]:
#Se generan los datos de entrenamiento y prueba
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test = train_test_split(X,y, test_size=0.2,random_state=42)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
from sklearn.svm import SVC

svc_model= SVC()
svc_model.fit(X_train, y_train)

In [ ]:
y_predict = svc_model.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
cm = confusion_matrix(y_test,y_predict)
sns.heatmap (cm, annot= True , fmt= "d")

In [ ]:
print (classification_report(y_test, y_predict))

In [ ]:
param_grid = {'C': [0.1,1,10,100,1000], 'gamma': [1,0.1,0.01,0.001,0.0001], 'kernel':['rbf']}


In [ ]:
from sklearn.model_selection import GridSearchCV

grid= GridSearchCV(SVC(),param_grid,refit= True,verbose =4)
grid.fit(X_train, y_train)

In [ ]:
grid.best_params_

In [ ]:
grid.best_estimator_

In [ ]:
grid_predictions = grid.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test,grid_predictions)
sns.heatmap(cm, annot=True, fmt= "d")

In [ ]:
print(classification_report(y_test,grid_predictions))